## Configure PUDL
The `.pudl.yml` configuration file tells PUDL where to look for data. Uncomment the next cell and run it if you're on our 2i2c JupyterHub.

In [1]:
#!cp ~/shared/shared-pudl.yml ~/.pudl.yml

In [2]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import logging
import os
import sys

import pandas as pd
import sqlalchemy as sa
import pudl

In [3]:
# setup for python logging
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

## Set your EIA API Key
Some of the routines in this notebook pull data from the EIA API to fill in missing fuel cost values. For them to work, you'll need to provide an API KEY.
* [Obtain an EIA API key here](https://www.eia.gov/opendata/register.php)
* If you put your API key in a shell environment variable named `API_KEY_EIA`, the next cell should work as it is (if you're using the `docker-compose.yml` in the `pudl-tutorial` repository to run our container yourself, it `API_KEY_EIA` will be passed in from your shell to the container automatically).
* You can also uncomment the first line of the next cell, and use it to set the `API_KEY_EIA` environment variable directly.
* If you're running this notebook on your own computer and want to learn more about setting environment variables outside of the notebook [see this blog post](https://www.twilio.com/blog/2017/01/how-to-set-environment-variables.html))

In [4]:
# os.environ["API_KEY_EIA"] = "put your API key here"
assert os.environ.get("API_KEY_EIA") is not None

# Using the PUDL output layer
The PUDL database tables are a clean, [normalized](https://en.wikipedia.org/wiki/Database_normalization) version of US electricity data. Normalized tables are great for databases and storage, but for interactive use, we often want a version of the data that includes plant and utility names and other associated info all in a single dataframe. These are "denormalized" tables. In addition to the referenced names and attributes like latitude and longitude or state, the denormalized tables often contain frequently calculated derived values (like calcuating `total_fuel_cost` from `total_heat_content_mmbtu` and `fuel_cost_per_mmbtu`). The Catalyst team developed a useful tool to access denormalized tables that we call the PUDL output object.

## What does the output layer provide?

Right now the output layer provides access to three different kinds of things:
 * denormalized tables
 * analytical outputs
 * partially integrated PUDL datasets that aren't in the database yet

## Why is the output layer useful?
Some benefits of using the output layer:
 * **Standardized denormalization:** You don't have to manually join the same tables together to get access to common attributes.
 * **Table caching:** many analyses rely on using the same table multiple times. The PUDL output object caches the tables in memory as pandas dataframes so you don't have to read tables from the database over and over again.
 * **Time series aggregation:** Some tables are annual, some monthly, some hourly. When you create a PUDL output object you can tell it to aggregate the data to either monthly or annual resolution for analysis.
 * **Standardized the filling-in of missing data:** There's a ton of missing or incomplete data. If requested, the output objects will use rolling averages and  data from the EIA API try to fill some of that missing data in.

# Instantiating Output Objects
* Grab the `pudl_settings`
* Create a connection to the PUDL Database
* Instantiate a `PudlTabl` object with that connection

In [5]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

{'pudl_in': '/home/zane/code/catalyst/pudl-work',
 'data_dir': '/home/zane/code/catalyst/pudl-work/data',
 'settings_dir': '/home/zane/code/catalyst/pudl-work/settings',
 'pudl_out': '/home/zane/code/catalyst/pudl-work',
 'sqlite_dir': '/home/zane/code/catalyst/pudl-work/sqlite',
 'parquet_dir': '/home/zane/code/catalyst/pudl-work/parquet',
 'ferc1_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite',
 'censusdp1tract_db': 'sqlite:////home/zane/code/catalyst/pudl-work/sqlite/censusdp1tract.sqlite'}

In [6]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pudl_engine

Engine(sqlite:////home/zane/code/catalyst/pudl-work/sqlite/pudl.sqlite)

In [7]:
# this configuration will return tables without aggregating by a time frequency... we'll explore that more below.
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## List the output object methods
* There are dozens of different data access methods within the `PudlTabl` object. If you want to see all of them with their docstrings, you can un-comment and run `help(pudl_out)` in the next cell.
* If you type `pudl_out.` and press `Shift` and `Tab` at the same time, you'll see a list of available methods as well.

In [8]:
#help(pudl_out)

This cell will print out a simple list of all the available public methods inside the `pudl_out` object

In [9]:
# this is the master list of all of the methods in the pudl_out object
# they all return a table cooresponding to their name
methods_pudl_out = [
    method_name for method_name in dir(pudl_out)
    if callable(getattr(pudl_out, method_name))    # if it is a method
    and '__' not in method_name                    # remove the internal methods
]
methods_pudl_out

['adjacency_ba_ferc714',
 'advanced_metering_infrastructure_eia861',
 'all_plants_ferc1',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'bf_eia923',
 'bga_eia860',
 'capacity_factor',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'demand_response_eia861',
 'demand_response_water_heater_eia861',
 'demand_side_management_ee_dr_eia861',
 'demand_side_management_misc_eia861',
 'demand_side_management_sales_eia861',
 'description_pa_ferc714',
 'distributed_generation_fuel_eia861',
 'distributed_generation_misc_eia861',
 'distributed_generation_tech_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'etl_eia861',
 'etl_ferc714',
 'fbp_ferc1',
 'frc_eia923',
 'fuel_cost',
 'fuel_ferc1',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gen_plants_ba_ferc714',
 'gens_eia860',
 'gf_eia923',
 'gfn_eia923',
 'green_pricing_eia861',
 'hr_by_gen',
 'hr_by_unit',
 'id_cer

## Basic Functionality

### Read a denormalized table
* Each of output object methods will return a Pandas Dataframe.
* Most of them correspond to a single database table, and will select all the data in that table, and automatically join it with some other useful information.
* Many of the access methods use an abbreviated name for the database table. E.g. the following reads all the data out of the `generators_eia860` table.

In [10]:
%%time
gens_eia860 = pudl_out.gens_eia860().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490824 entries, 490823 to 0
Columns: 111 entries, report_date to zip_code
dtypes: Int64(8), boolean(28), category(1), datetime64[ns](11), float64(19), int64(1), object(1), string(42)
memory usage: 341.2+ MB
CPU times: user 1min 16s, sys: 6.87 s, total: 1min 23s
Wall time: 1min 30s


### Automatic dataframe caching
The `generators_eia860` table is quite long, and the above cell probably took several seconds to read 270,000 records each with 100 columns, creating an 800MB Dataframe. If you run the same output routine again, it will complete almost instantly because that dataframe is already stored inside `pudl_out`. This is memory intensive, but can save time in calculations that need to use the same tables several times.

In [11]:
%%time
gens_again_eia860 = pudl_out.gens_eia860()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.68 µs


## Exploring `pudl_out` Arguments
Below, we'll explore the main arguments that are used to customize the PUDL output object. You can mix and match these options.

By default, the output object will read data from all available years, do no time aggregation, and not attempt to fill in missing values.

In [12]:
# here are the default arguments for the pudl_out object
pudl_out = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq=None,               # Desired time grouping to aggregate PUDL tables to.
    start_date=None,         # Beginning date for data to pull from the PUDL DB.
    end_date=None,           # End date for data to pull from the PUDL DB.
    fill_fuel_cost=False,    # Whether to fill in missing fuel costs with EIA monthly state-level averages.
    roll_fuel_cost=False,    # Whether to fill in monthly missing fuel costs with a 12-month rolling average.
)

### Time series aggregation
The PUDL output object can aggregate data on a monthly or annual basis, if you set the `freq` argument to `AS` (annual starting at the beginning of the calendar year) or `MS` (monthly starting at the beginning of the month) or [other equivalent frequency abbreviations](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases).

**NOTE:** Not all columns can be aggregated, so you may lose access to some kinds of information in aggregated outputs. If you need to retain information that gets lost in the default aggregation / groupby process, you may need to pull the unaggregated data and do your own aggregation.

In [13]:
pudl_out_as = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='AS',               # Aggregate tables annually
)

In [14]:
pudl_out_as.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh,unit_id_pudl
0,2008-01-01,3,32,Barry,195,18,Alabama Power Co,1,873997.0,NaN
1,2008-01-01,3,32,Barry,195,18,Alabama Power Co,2,813505.0,NaN
2,2008-01-01,3,32,Barry,195,18,Alabama Power Co,3,1458889.0,NaN
3,2008-01-01,3,32,Barry,195,18,Alabama Power Co,4,2445624.0,NaN
4,2008-01-01,3,32,Barry,195,18,Alabama Power Co,5,3968109.0,NaN


In [15]:
pudl_out_ms = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
)

In [16]:
pudl_out_ms.gen_eia923().head()

,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh,unit_id_pudl
0,2008-01-01,3,32,Barry,195,18,Alabama Power Co,1,96021.0,NaN
1,2008-01-01,3,32,Barry,195,18,Alabama Power Co,2,92534.0,NaN
2,2008-01-01,3,32,Barry,195,18,Alabama Power Co,3,161559.0,NaN
3,2008-01-01,3,32,Barry,195,18,Alabama Power Co,4,257675.0,NaN
4,2008-01-01,3,32,Barry,195,18,Alabama Power Co,5,182397.0,NaN


### Filling in Missing Fuel Costs
 * The original EIA data is often incomplete.
 * Many utilities withold information about their fuel costs.
 * We have a couple of ways of estimating missing values, if you need complete data.

The ouput object created in the next cell will attempt to use all of these methods to fill in missing data.
To fill in missing fuel costs, we can pull monthly state-level average fuel costs from EIA, and we can use rolling averages to fill in short gaps in the data.
* Set `fill_fuel_cost=True` when creating an output object to pull average monthly fuel costs from the EIA API.
* Set `roll_fuel_cost=True` when creating an output object to use a 12-month rolling average based on available data to fill in gaps.
* These options can be used together to fill in as many gaps as possible.
* **NOTE:** You need to have set the `API_KEY_EIA` environment variable to a valid EIA API key for this to work. See instructions at the top of this notebook.

In [17]:
pudl_out_fill = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine, # we always need a pudl_engine
    freq='MS',               # Aggregate tables monthly
    fill_fuel_cost=True,     # Fill in missing fuel cost records with state-level averages from EIA's API
    roll_fuel_cost=True,     # Fill in missing fuel cost records with a 12-month rolling average.
)

In [18]:
%%time
pudl_out_fill.frc_eia923().head()

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 3min 2s, sys: 1.4 s, total: 3min 3s
Wall time: 3min 10s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,fuel_consumed_mmbtu,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,fuel_mmbtu_per_unit,fuel_received_units,fuel_type_code_pudl,mercury_content_ppm,moisture_content_pct,sulfur_content_pct,total_fuel_cost
0,2008-01-01,3,32,Barry,195,18,Alabama Power Co,5.450288,NaN,7183512.000,False,2.131684,23.049712,311653.0,coal,NaN,NaN,0.488324,1.531298e+07
1,2008-01-01,3,32,Barry,195,18,Alabama Power Co,0.000000,NaN,2892180.141,False,8.631000,1.039000,2783619.0,gas,NaN,NaN,0.000000,2.496241e+07
2,2008-01-01,7,207,Gadsden,195,18,Alabama Power Co,14.720584,NaN,658516.845,False,2.780174,24.604575,26764.0,coal,NaN,NaN,1.662357,1.830791e+06
3,2008-01-01,7,207,Gadsden,195,18,Alabama Power Co,0.000000,NaN,2385.479,False,7.260000,1.019000,2341.0,gas,NaN,NaN,0.000000,1.731858e+04
4,2008-01-01,8,231,Gorgas,195,18,Alabama Power Co,14.475849,NaN,8122181.817,False,2.448640,23.770894,341686.0,coal,NaN,NaN,1.710048,1.988830e+07


Looking at the filled vs. unfilled monthly data in the Fuel Receipts and Costs data from EIA 923, we can see that there are about 190k possible monthly records. Unfilled, we have fuel costs for about 107k of them. With the state level monthly fuel costs and rolling averages, we can get that up to about 116k records. An improvement, but it's not great. Unfortunately this data simply isn't reported publicly.

In [19]:
pudl_out_ms.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230063 entries, 0 to 231966
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         230063 non-null  Int64         
 1   report_date          230063 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  127785 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 7.2 MB


In [20]:
pudl_out_fill.frc_eia923()[["plant_id_eia", "report_date", "fuel_cost_per_mmbtu"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230063 entries, 0 to 231966
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   plant_id_eia         230063 non-null  Int64         
 1   report_date          230063 non-null  datetime64[ns]
 2   fuel_cost_per_mmbtu  207961 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(1)
memory usage: 7.2 MB


## Free Memory
Because this JupyterHub has limited memory, we need to delete the cached dataframes when we're done with them.

In [21]:
del pudl_out
del pudl_out_ms
del pudl_out_as
del pudl_out_fill

# Denormalized Output Tables
* Below, we'll extract and show a sample of many of the available denormalized PUDL output tables.
* If you'd like to see more than 5 sample rows, feel free to change `n_samples` below.
* Rather than assigning the results of these functions to a local variable in the notebook, we're showing samples from the cached dataframes to conserve memory, as this JupyterHub has limited RAM available at the moment.

In [22]:
n_samples = 5
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

## EIA Forms 860 & 923

In [23]:
# here are all of the EIA tables
tables_eia = [
    t for t in methods_pudl_out 
    if '_eia' in t 
    and '_eia861' not in t       # avoid the EIA 861 tables for now bc it is preliminary
]
tables_eia

['bf_eia923',
 'bga_eia860',
 'frc_eia923',
 'gen_allocated_eia923',
 'gen_eia923',
 'gen_original_eia923',
 'gens_eia860',
 'gf_eia923',
 'gfn_eia923',
 'own_eia860',
 'plants_eia860',
 'pu_eia860',
 'utils_eia860']

### EIA Plant Utility Associations
This is mostly a helper function, used for adding plant and utility names and IDs into other output tables.

In [24]:
%%time
pudl_out.pu_eia860().sample(n_samples)

CPU times: user 6.93 s, sys: 76 ms, total: 7 s
Wall time: 7.01 s


,report_date,plant_id_eia,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl
952,2017-01-01,78,Snettisham,1311,213,Alaska Electric Light&Power Co,19
159571,2017-01-01,61439,Panasonic Carport Solar Hybrid,11056,15466,Public Service Co of Colorado,272
43225,2006-01-01,4080,Petenwell,1055,20863,Wisconsin River Power Co,366
68914,2014-01-01,10623,Civic Center,3316,11216,Los Angeles County,2300
58667,2015-01-01,7876,APC2,9651,195,Alabama Power Co,18


### EIA 860 Boiler Generator Associations
* **NOTE:** We have filled in many more boiler-generator associations based on additional information. The `bga_source` column indicates where the association came from.

In [25]:
%%time
pudl_out.bga_eia860().sample(n_samples)

CPU times: user 788 ms, sys: 3.97 ms, total: 792 ms
Wall time: 796 ms


,plant_id_eia,report_date,generator_id,boiler_id,unit_id_eia,unit_id_pudl,bga_source
11997,1252,2011-01-01,8,10,None,2,eia860_org
9929,50969,2010-01-01,GEN4,HRSG1,None,1,eia860_org
91616,56407,2015-01-01,3ST,3A,WC3,3,eia860_org
86979,58007,2014-01-01,G5,HRSG3,CC1,1,eia860_org
3705,50476,2009-01-01,GEN2,B9,None,1,eia860_org


### EIA 860 Plants

In [26]:
%%time
pudl_out.plants_eia860().sample(n_samples)

CPU times: user 3.75 s, sys: 36.1 ms, total: 3.79 s
Wall time: 3.79 s


,plant_id_eia,plant_name_eia,balancing_authority_code_eia,balancing_authority_name_eia,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,...,respondent_frequency,service_area,transmission_distribution_owner_id,transmission_distribution_owner_name,transmission_distribution_owner_state,utility_id_eia,water_source,plant_id_pudl,utility_name_eia,utility_id_pudl
113352,56189,"Waterside Power, LLC",ISNE,ISO New England Inc.,Stamford,Fairfield,False,True,False,115.0,...,None,Connecticut Light & Power Co,NaN,None,None,49840,Aquarian Water Co,4832,"Waterside Power, LLC",3712
160420,61449,RE Gaskell West 5 LLC,CISO,California Independent System Operator,Rosamond,Kern,False,False,False,230.0,...,None,None,17609.0,Southern California Edison Co,CA,61069,None,11065,RE Gaskell West LLC,7583
28088,2408,Mercer,PJM,"PJM Interconnection, LLC",Hamilton Township,Mercer,False,True,False,230.0,...,None,None,15477.0,Public Service Elec & Gas Co,NJ,15147,Delaware River,1174,PSEG Fossil LLC,268
109037,55843,"Bonnet Carre' Power, LLC",None,None,La Place,St John The Baptist,False,True,False,NaN,...,None,Entergy Louisiana Inc,NaN,None,None,1940,Mississippi River,13213,Bonnet Carre' Power LLC,6842
102506,55394,"Cin Cap IX, LLC",None,None,None,Kenton,False,True,False,NaN,...,None,Union Light Heat & Power Co,NaN,None,None,3559,None,13439,Cinergy Capital & Trading Inc,7665


### EIA 860 Generators

In [27]:
%%time
pudl_out.gens_eia860().sample(n_samples)

CPU times: user 52.4 s, sys: 5.23 s, total: 57.7 s
Wall time: 58.2 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,associated_combined_heat_power,balancing_authority_code_eia,...,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,unit_id_pudl,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
475112,2001-01-01,55109,4380,Rocky Road Power LLC,16197,3038,Rocky Road Power LLC,TG3,False,PJM,...,NaN,<NA>,<NA>,<NA>,NaT,<NA>,True,40.67,NaN,60118
309013,2010-01-01,431,1448,Jaybird,16534,3069,Sacramento Municipal Util Dist,H1,False,BANC,...,NaN,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,78.10,NaN,95725
347229,2008-01-01,3029,1005,Lemolo 1,14354,246,PacifiCorp,1,False,PACW,...,NaN,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,29.00,NaN,97447
411120,2005-01-01,1769,466,Presque Isle,20847,363,Wisconsin Electric Power Co,4,False,MISO,...,NaN,0,<NA>,<NA>,NaT,<NA>,<NA>,58.00,NaN,49855
303012,2010-01-01,3178,7908,Armstrong,23279,448,Allegheny Energy Supply Co LLC,2,False,PJM,...,NaN,0,<NA>,2,NaT,<NA>,<NA>,176.00,NaN,16210


### EIA 860 Generator-level Ownership

In [28]:
%%time
pudl_out.own_eia860().sample(n_samples)

CPU times: user 7.53 s, sys: 105 ms, total: 7.64 s
Wall time: 7.77 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,owner_utility_id_eia,owner_name,fraction_owned,owner_city,owner_state,owner_street_address,owner_zip_code
44078,2013-01-01,7777,2882,Dover Peaking,40577,484,American Mun Power-Ohio Inc,1,99999,Other,0.27,None,None,None,None
60240,2016-01-01,60231,9186,Genentech-Oceanside Hybrid,60947,5629,Tesla Inc.,PV1,59865,"Dom Solar Lessor I, LP",1.00,San Mateo,CA,None,94402
25648,2008-01-01,54268,4018,March Point Cogen,11459,2349,March Point Cogeneration Co,GTG1,34505,Edison Mission Energy,0.50,None,None,None,None
61604,2017-01-01,6664,353,Louisa,12341,185,MidAmerican Energy Co,1,9417,Interstate Power and Light Co,0.04,Cedar Rapids,IA,200 First Street SE,52401
36678,2011-01-01,54462,4093,Graniteville Co Enterprise Div,55810,549,Augusta Canal Authority,2,22228,Enterprise Mill LLC,1.00,None,GA,None,None


### EIA 923 Generation and Fuel Consumption

In [29]:
%%time
pudl_out.gf_eia923().sample(n_samples)

CPU times: user 28.9 s, sys: 1.73 s, total: 30.6 s
Wall time: 34.3 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,energy_source_code,fuel_consumed_for_electricity_mmbtu,fuel_consumed_for_electricity_units,fuel_consumed_mmbtu,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code_aer,fuel_type_code_pudl,net_generation_mwh,prime_mover_code
1978882,2017-10-01,57204,5538,PSEG Hackettstown,56584,2916,PSEG Solar Hackettstown LLC,SUN,1359.00,0.00,1359.00,0.00,0.000,SUN,solar,147.524,PV
475139,2005-11-01,6089,340,Lewis & Clark,12199,178,MDU Resources Group Inc,NG,732.28,634.01,732.28,634.01,1.155,NG,gas,29.189,ST
977639,2010-04-01,10641,3324,Cambria Cogen,2884,773,Cambria CoGen Co,WC,347570.00,23018.00,366673.00,24283.00,15.100,WOC,coal,28225.000,ST
290455,2004-03-01,173,503,Robert E Ritchie,814,106,Entergy Arkansas LLC,RFO,0.00,0.00,0.00,0.00,0.000,RFO,oil,-151.000,ST
2495048,2020-12-01,50316,3581,Central Soya Co Inc,3283,729,Central Soya Co Inc,BIT,0.00,0.00,0.00,0.00,0.000,COL,coal,0.000,ST


### EIA 923 Fuel Receipts and Costs

In [30]:
%%time
pudl_out.frc_eia923().sample(n_samples)

CPU times: user 23.3 s, sys: 48.3 ms, total: 23.4 s
Wall time: 23.4 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,ash_content_pct,chlorine_content_ppm,contract_expiration_date,...,mine_state,mine_type,moisture_content_pct,natural_gas_delivery_contract_type_code,natural_gas_transport_code,primary_transportation_mode_code,secondary_transportation_mode_code,sulfur_content_pct,supplier_name,total_fuel_cost
61090,2008-12-01,54693,4188,York Cogen Facility,18458,4406,York Generation Company LLC,0.0,NaN,NaT,...,NaN,NaN,NaN,,firm,<NA>,<NA>,0.00,hess,NaN
320223,2014-02-01,3396,2306,Bull Run,18642,3430,Tennessee Valley Authority,7.0,NaN,2013-12-01,...,IN,underground,NaN,,,RR,<NA>,1.58,alliance coal,841231.440000
531114,2019-11-01,6034,44,Belle River,5109,89,The DTE Electric Company,0.0,NaN,NaT,...,NaN,NaN,NaN,firm,firm,PL,<NA>,0.00,various (natural gas spot purchases only),140366.909380
508854,2019-04-01,6204,2558,Laramie River,1307,582,Basin Electric Power Coop,5.0,0.0,2022-12-01,...,WY,surface,26.73,,,RR,TR,0.26,arch,284695.064368
295518,2013-06-01,6113,222,Gibson,15470,92,PSI Energy Inc,0.0,NaN,2014-04-01,...,NaN,NaN,NaN,,,TR,<NA>,0.00,hightower petroleum,712986.102250


### EIA 923 Boiler Fuel Consumption

In [31]:
%%time
pudl_out.bf_eia923().sample(n_samples)

CPU times: user 22.7 s, sys: 705 ms, total: 23.4 s
Wall time: 24.8 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,boiler_id,unit_id_pudl,ash_content_pct,energy_source_code,fuel_consumed_mmbtu,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type_code_pudl,sulfur_content_pct
300142,2011-09-01,50881,3821,Shasta,20541,3763,Wheelabrator Environmental Sys,BLR1,1,0.0,AB,0.000,0.0,0.000,waste,0.0
1167919,2019-09-01,2866,2191,W H Sammis,6526,1729,FirstEnergy,4,4,0.0,BIT,0.000,0.0,0.000,coal,0.0
1133818,2019-05-01,10686,477,Rapids Energy Center,12647,23,Minnesota Power Inc,7,1,0.0,NG,43.000,43.0,1.000,gas,0.0
1103329,2019-02-01,1927,496,Riverside (MN),13781,224,Northern States Power Co,10,2,0.0,NG,1168859.325,1087311.0,1.075,gas,0.0
322706,2011-12-01,55340,4500,TPS Dell LLC,924,539,Associated Electric Coop Inc,HRSG1,1,0.0,NG,95.697,93.0,1.029,gas,0.0


### EIA 923 Net Generation by Generator

In [32]:
%%time
pudl_out.gen_eia923().sample(n_samples)

CPU times: user 14.1 s, sys: 129 ms, total: 14.2 s
Wall time: 14.3 s


,report_date,plant_id_eia,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,generator_id,net_generation_mwh,unit_id_pudl
384965,2017-04-01,50231,3538,Gorge Energy Div SDS Lumber Co,22803,3108,SDS Lumber Co,TG3,0.0,1.0
246671,2014-07-01,3006,2223,Anadarko,20447,3737,Western Farmers Elec Coop Inc,1,0.0,1.0
320882,2015-12-01,55062,4351,Tenaska Frontier,18611,3425,Tenaska Frontier Partners Ltd,GTG1,108303.0,1.0
546739,2020-09-01,54512,4099,Domino Sugar Arabi,547,486,The American Sugar Refining Co,TG2,0.0,1.0
168917,2012-10-01,983,114,Clifty Creek,9269,145,Indiana-Kentucky Electric Corp,1,75552.0,1.0


## FERC Form 1
* Only a small subset of the 100+ tables that exist in the original FERC Form 1 have been cleaned and included in the PUDL DB.
* For tables not included here, you'll need to access the cloned multi-year FERC 1 DB that we produce. See the first tutorial notebook for more information.

In [33]:
# All of the FERC Form 1 tables end with _ferc1
tables_ferc1 = [
    t for t in methods_pudl_out 
    if '_ferc1' in t 
]
tables_ferc1

['all_plants_ferc1',
 'fbp_ferc1',
 'fuel_ferc1',
 'plant_in_service_ferc1',
 'plants_hydro_ferc1',
 'plants_pumped_storage_ferc1',
 'plants_small_ferc1',
 'plants_steam_ferc1',
 'pu_ferc1',
 'purchased_power_ferc1']

### FERC 1 Large Steam Plants
The large steam plants report detailed operating expenses in this table, as well as operational characteristics.

In [34]:
%%time
pudl_out.plants_steam_ferc1().sample(n_samples)

CPU times: user 618 ms, sys: 24.3 ms, total: 642 ms
Wall time: 767 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_id_ferc1,plant_name_ferc1,asset_retirement_cost,avg_num_employees,capacity_factor,...,opex_steam,opex_steam_other,opex_structures,opex_transfer,peak_demand_mw,plant_capability_mw,plant_hours_connected_while_generating,plant_type,record_id,water_limited_capacity_mw
22360,2014,44,89,DTE Electric Company,587,180,trenton channel pp,NaN,144.0,0.375364,...,1442844.0,NaN,2377250.0,NaN,612.0,730.0,8636.0,steam,f1_steam_2014_12_44_1_2,675.0
19228,2011,120,224,Northern States Power Company (Minnesota),286,918,inver hills,NaN,4.0,0.010626,...,NaN,NaN,215894.0,NaN,301.0,372.0,376.0,combustion_turbine,f1_steam_2011_12_120_2_3,282.0
27682,2020,145,272,Public Service Company of Colorado,108,1764,"cherokee 5, 6, & 7",NaN,NaN,0.537652,...,NaN,NaN,972989.0,NaN,788.0,600.0,8385.0,combined_cycle,f1_steam_2020_12_145_0_3,576.0
22656,2014,195,365,Wisconsin Public Service Corporation,343,972,"w marinette m31, m32",NaN,3.0,0.011374,...,NaN,NaN,75.0,NaN,NaN,NaN,298.0,combustion_turbine,f1_steam_2014_12_195_3_2,76.0
16374,2008,108,204,"Nevada Power Company, d/b/a NV Energy",566,765,"sun peak 3, 4, 5",NaN,NaN,0.006268,...,NaN,NaN,NaN,NaN,NaN,222.0,NaN,combustion_turbine,f1_steam_2008_12_108_1_3,NaN


### FERC 1 Fuel
Fuel consumption by the large steam plants, broken down by plant and fuel type.

In [35]:
%%time
pudl_out.fuel_ferc1().sample(n_samples)

CPU times: user 314 ms, sys: 7.84 ms, total: 322 ms
Wall time: 387 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,fuel_consumed_mmbtu,fuel_consumed_total_cost,fuel_consumed_units,fuel_cost_per_mmbtu,fuel_cost_per_unit_burned,fuel_cost_per_unit_delivered,fuel_mmbtu_per_unit,fuel_type_code_pudl,fuel_units,record_id
4130,2004,193,363,Wisconsin Electric Power Company,466,presque isle-total,1.180037e+05,9.663439e+05,20286.0,8.1902,47.636,47.636,5.817000,oil,bbl,f1_fuel_2004_12_193_0_7
28921,2012,182,161,KCP&L Greater Missouri Operations Company,307,jeffrey ener ctr 8%,1.104600e+07,1.936850e+07,660883.0,1.7530,29.307,30.476,16.714000,coal,ton,f1_fuel_2012_12_182_0_7
3716,2002,87,107,"Entergy Louisiana, LLC",618,waterford 3,9.405858e+07,4.313476e+07,27562146.0,0.4590,1.565,0.000,3.412600,nuclear,mwhth,f1_fuel_2002_12_87_0_2
1473,1996,8,106,"Entergy Arkansas, Inc.",344,mabelvale,6.417015e+03,3.696310e+04,1086.0,5.7600,34.036,34.036,5.908854,oil,bbl,f1_fuel_1996_12_8_2_8
20223,2000,149,276,Public Service Electric and Gas Company,1174,mercer (steam),1.136803e+06,4.539456e+06,1107995.0,1.5170,4.097,4.097,1.026000,gas,mcf,f1_fuel_2000_12_149_2_8


### FERC 1 Fuel by Plant
Wide-form aggregated fuel totals by plant and year, identifying the relative cost and heat content proportions of different fuels, as well as the primary fuel for the plant.

In [36]:
%%time
pudl_out.fbp_ferc1().sample(n_samples)

CPU times: user 400 ms, sys: 8 ms, total: 408 ms
Wall time: 413 ms


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,plant_id_pudl,plant_name_ferc1,coal_fraction_cost,coal_fraction_mmbtu,fuel_cost,fuel_mmbtu,...,nuclear_fraction_cost,nuclear_fraction_mmbtu,oil_fraction_cost,oil_fraction_mmbtu,other_fraction_cost,other_fraction_mmbtu,primary_fuel_by_cost,primary_fuel_by_mmbtu,waste_fraction_cost,waste_fraction_mmbtu
7378,2018,81,162,Kentucky Power Company,382,mitchell- total,0.964745,0.994338,1.497813e+08,5.748272e+07,...,0.0,0.0,0.035255,0.005662,0.0,0.0,coal,coal,0.0,0.0
347,2010,6,29,Appalachian Power Company,447,philip sporn - apco,0.982111,0.996390,4.015101e+07,1.224574e+07,...,0.0,0.0,0.017889,0.003610,0.0,0.0,coal,coal,0.0,0.0
4528,1997,51,318,The Empire District Electric Company,183,energy center,0.000000,0.000000,2.619139e+06,9.952528e+05,...,0.0,0.0,0.000000,0.000000,0.0,0.0,gas,gas,0.0,0.0
79,1995,2,18,ALABAMA POWER COMPANY,109,chickasaw,0.000000,0.000000,5.969851e+05,2.837261e+05,...,0.0,0.0,0.000000,0.000000,0.0,0.0,gas,gas,0.0,0.0
9636,2002,120,224,Northern States Power Company (Minnesota),18,angus anson,0.000000,0.000000,3.480281e+06,9.934929e+05,...,0.0,0.0,0.000421,0.000253,0.0,0.0,gas,gas,0.0,0.0


### FERC 1 Plant in Service
An accounting of how much electric plant infrastructure exists in each of the many FERC accounts. This is a very wide form table.

In [37]:
%%time
pudl_out.plant_in_service_ferc1().sample(n_samples)

CPU times: user 1.33 s, sys: 19 ms, total: 1.35 s
Wall time: 1.52 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,record_id,amount_type,distribution_acct360_land,distribution_acct361_structures,distribution_acct362_station_equip,distribution_acct363_storage_battery_equip,...,transmission_acct352_structures,transmission_acct353_station_equip,transmission_acct354_towers,transmission_acct355_poles,transmission_acct356_overhead_conductors,transmission_acct357_underground_conduit,transmission_acct358_underground_conductors,transmission_acct359_1_asset_retirement,transmission_acct359_roads_trails,transmission_total
7458,2002,72,145,Indiana-Kentucky Electric Corporation,f1_plant_in_srvce_2002_12_72_0,additions,NaN,NaN,NaN,NaN,...,10086.0,299960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310046.0
2104,2016,19,55,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,f1_plant_in_srvce_2016_12_19_0,transfers,NaN,NaN,3600.0,NaN,...,NaN,-3600.0,NaN,278237.0,-278237.0,NaN,NaN,NaN,NaN,-3600.0
22281,1995,212,38,BeeBee Island Corporation,f1_plant_in_srvce_1995_12_212_0,starting_balance,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10377,2006,99,190,Mississippi Power Company,f1_plant_in_srvce_2006_12_99_0,retirements,2866.0,133399.0,3987857.0,NaN,...,11155.0,597750.0,492088.0,472390.0,1617839.0,NaN,NaN,NaN,NaN,3191222.0
5001,1994,46,98,Duquesne Light Company,f1_plant_in_srvce_1994_12_46_0,transfers,NaN,-581.0,-91765.0,521.0,...,NaN,-301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-301.0


### FERC 1 Purchased Power
A summary of electricity market transactions between utilities. Sadly the sellers are identified only by their names, and not their FERC Utility (Respondent) ID.

In [38]:
%%time
pudl_out.purchased_power_ferc1().sample(n_samples)

CPU times: user 1.5 s, sys: 31.9 ms, total: 1.54 s
Wall time: 1.98 s


,report_year,utility_id_ferc1,utility_id_pudl,utility_name_ferc1,seller_name,record_id,billing_demand_mw,coincident_peak_demand_mw,delivered_mwh,demand_charges,energy_charges,non_coincident_peak_demand_mw,other_charges,purchase_type,purchased_mwh,received_mwh,tariff,total_settlement
179732,2009,19,55,CENTRAL HUDSON GAS & ELECTRIC CORPORATION,Florida Power & Light,f1_purchased_pwr_2009_12_19_2_8,NaN,NaN,0.0,0.0,195152.0,NaN,0.0,other_service,0.0,0.0,,195152.0
117472,2017,159,292,South Carolina Electric & Gas Company,"Duke Energy Carolinas, LLC",f1_purchased_pwr_2017_12_159_1_6,NaN,NaN,0.0,0.0,94051.0,NaN,0.0,other_service,2534.0,0.0,,94051.0
151624,2014,120,224,Northern States Power Company (Minnesota),South Ridge,f1_purchased_pwr_2014_12_120_6_3,NaN,NaN,0.0,0.0,258714.0,NaN,0.0,long_unit,7840.0,0.0,,258714.0
5008,1998,7,31,Arizona Public Service Company,Los Alamos County,f1_purchased_pwr_1998_12_7_3_2,NaN,NaN,0.0,0.0,1720.0,NaN,0.0,other_service,31.0,0.0,,1720.0
94430,2019,161,295,Southern California Edison Company,"US TOPCO ENERGY, INC (SOCCER CENTER)",f1_purchased_pwr_2019_12_161_27_11,NaN,NaN,0.0,0.0,554864.0,NaN,0.0,other_service,6622.0,0.0,,554864.0


## Free Memory
Again, because we're on a JupyterHub with limited RAM per user, we need to delete the cached dataframes we've just created.

In [39]:
del pudl_out

# Analysis Outputs
* The PUDL Database is mainly meant to standardize the structure of data that's been reported in different ways over different years, so that it can all be used together.
* We typically don't include calculated values or big modifications to the original data.
* We're compiling a growing library of stock analyses in the `pudl.analysis` subpackage, which operate on data stored in the database.
* Some of these analytical outputs are build into the output object so that they can take advantage of the dataframe caching, and for convenient access.

## The Marginal Cost of Electricity (MCOE)
* One of our first analysis modules calculates fuel costs, heat rates, and capacity factors on a generator by generator basis.
* The long term goal is for it to provide a comprehensive marginal cost of electricity production (MCOE).
* The integration of operating costs from FERC Form 1 is still a work in progress, and hasn't been added in here yet.

### MCOE Requires Aggregation
* Fuel costs and other data need to be aggregated by month or year to calculate MCOE.
* This means we need an output object that aggregates by month or year.
* Because a single `NA` value can wipe out a whole aggregated category, you'll get more information with a Monthly aggregation.

In [40]:
pudl_out_monthly = pudl.output.pudltabl.PudlTabl(
    pudl_engine=pudl_engine,
    freq="MS",
    fill_fuel_cost=True,
    roll_fuel_cost=True,
)

### Heat Rate by Generation Unit (MMBTU/MWh)
* A "Generation Unit" (identifyed by `unit_id_pudl` here) is a group of "boilers" (where fuel is consumed) and "generators" (where electricity is made) which are connected to each other.
* Because the fuel inputs and electricity outputs are comingled, this is the most granular level at which a direct heat rate calculation can be done.

In [41]:
%%time
pudl_out_monthly.hr_by_unit().sample(n_samples)

CPU times: user 9min 9s, sys: 694 ms, total: 9min 9s
Wall time: 9min 19s


,report_date,plant_id_eia,unit_id_pudl,net_generation_mwh,fuel_consumed_mmbtu,heat_rate_mmbtu_mwh
38027,2009-11-01,50110,1,6518.0,152640.000,23.418226
312982,2019-10-01,10869,1,19391.0,354494.064,18.281371
124994,2012-11-01,55343,1,44935.0,0.000,0.000000
120124,2012-09-01,55380,3,59838.0,64174.225,1.072466
172809,2014-07-01,2836,2,262481.0,NaN,NaN


### Heat Rate by Generator (mmBTU/MWh)
* However, we do need per-generator heat rates to estimate per-generator fuel costs.

In [42]:
%%time
pudl_out_monthly.hr_by_gen().sample(n_samples)

CPU times: user 1min 18s, sys: 4.71 s, total: 1min 23s
Wall time: 1min 23s


,report_date,plant_id_eia,unit_id_pudl,generator_id,heat_rate_mmbtu_mwh,fuel_type_code_pudl,fuel_type_count
214482,2014-06-01,54427,1,NO9,NaN,waste,1
396311,2018-05-01,1893,1,1,11.341448,coal,2
134036,2012-06-01,50296,1,GEN2,42.566338,waste,1
423915,2018-12-01,7870,1,CTG2,9.786825,gas,1
315757,2016-08-01,728,1,1,NaN,coal,2


### Per-generator Fuel Costs
* Calculate per-generator fuel costs based on heat rates and fuel deliveries
* Because we told the `pudl_out` object to try and fill in missing values, this will request monthly average fuel cost data by date from the EIA API. It might take a minute.
* This also means you'll need to have set your EIA API Key at the top of the notebook.

In [43]:
%%time
pudl_out_monthly.fuel_cost().sample(n_samples)

filling in fuel cost NaNs EIA APIs monthly state averages
filling in fuel cost NaNs with rolling averages
CPU times: user 3min 33s, sys: 694 ms, total: 3min 33s
Wall time: 3min 43s


,plant_id_eia,generator_id,unit_id_pudl,report_date,plant_name_eia,plant_id_pudl,utility_id_eia,utility_name_eia,utility_id_pudl,fuel_type_count,fuel_type_code_pudl,fuel_cost_from_eiaapi,fuel_cost_per_mmbtu,heat_rate_mmbtu_mwh,fuel_cost_per_mwh
406427,55357,CTG2,1,2018-07-01,Brazos Valley Generating Facil,4508,2171,Brazos Valley Energy,692,1,gas,True,2.910156,7.459330,21.707817
441478,990,6,4,2019-05-01,Elmer W Stout,259,9273,Indianapolis Power & Light Co,146,3,gas,False,3.587000,12.946987,46.440841
16116,8,10,1,2009-02-01,Gorgas,231,195,Alabama Power Co,18,1,coal,False,3.801707,9.707142,36.903709
286447,10865,GEN6,1,2015-12-01,Decatur,3421,772,Archer Daniels Midland Co,522,1,coal,True,NaN,35.378351,NaN
504701,3797,CT8,6,2020-10-01,Chesterfield,111,19876,Dominion Virginia Power,349,2,gas,False,3.140000,7.347615,23.071513


### Per-generator Capacity Factor

In [44]:
%%time
pudl_out_monthly.capacity_factor().sample(n_samples)

CPU times: user 3.5 s, sys: 39 µs, total: 3.5 s
Wall time: 3.54 s


,plant_id_eia,report_date,generator_id,net_generation_mwh,capacity_mw,capacity_factor
537457,3453,2020-07-01,6,14414.0,135.7,0.142768
536880,963,2020-07-01,4,76082.0,230.1,0.444419
462439,58084,2018-11-01,GT100,10457.0,16.1,0.902088
120826,1710,2011-06-01,3,466280.0,916.8,0.706382
45381,8223,2009-04-01,1,57794.0,424.8,0.188958


### Per-generator MCOE
* This function uses the cached dataframes that were generated above to produce a huge table of per-generator statistics.
* If you just called this function alone, all of those other dataframes would be automatically generated, and available within the output object.

In [45]:
%%time
pudl_out_monthly.mcoe().sample(n_samples)

CPU times: user 1min 3s, sys: 8.91 s, total: 1min 12s
Wall time: 1min 14s


,plant_id_eia,generator_id,report_date,unit_id_pudl,plant_id_pudl,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,associated_combined_heat_power,...,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,ultrasupercritical_tech,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_estimate,winter_capacity_mw,winter_estimated_capability_mw,zip_code
350643,56686,26,2011-01-01,<NA>,5188,King & Queen,55938,2086,Industrial Power Generating Company LLC,False,...,NaN,NaN,<NA>,<NA>,NaT,<NA>,<NA>,0.3,NaN,23091
249287,672,19,2010-01-01,<NA>,7730,Hansel,10376,2214,Kissimmee Utility Authority,False,...,NaN,NaN,0,<NA>,NaT,<NA>,<NA>,2.0,NaN,34741
712109,7238,4A,2017-02-01,3,2745,Cane Island,10376,2214,Kissimmee Utility Authority,False,...,3.792420e+05,NaN,<NA>,<NA>,NaT,False,<NA>,140.0,NaN,33848
981892,6021,3,2021-01-01,<NA>,140,Craig,30151,3514,Tri-State G & T Assn Inc,False,...,NaN,NaN,<NA>,<NA>,NaT,<NA>,<NA>,448.0,NaN,81626
779277,6068,2,2018-03-01,2,307,Jeffrey,22500,359,Western Resources Inc,False,...,1.711682e+06,NaN,<NA>,<NA>,NaT,False,<NA>,733.0,NaN,66536


## Free Memory

In [46]:
del pudl_out_monthly

# Preliminary Output Tables 
* Integrating a new dataset into the PUDL database requires many steps (datastore, extract, transform, load, outputs).
* Sometimes we need to use tables from new datasets as soon as possible for analysis.
* The interim extract and transform steps can be hacked into the output object to run on the fly, prior to DB integration.
* The data extraction and transformation can take a while though -- and it will need to be re-run from scratch every time you create a new output object.
* **WARNING:** None of this data has been fully validated, and the structure is likely to change. Some of it (especially the FERC 714) is still in a pretty raw state.

As of December 2020, we have preliminarily integrated EIA 861 and FERC 714 in this format.

## EIA Form 861
* The interim EIA 861 ETL is set up to automatically run in its entirety as soon as you request any EIA 861 table.
* This should take 2-5 minutes if you already have the raw input data avaialble.
* If raw input data needs to be downloaded [from our Zenodo archives](https://zenodo.org/record/4127029) first (which should happen automatically), it will take longer.

In [47]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [48]:
# here are all of the EIA 861 tables
methods_eia861 = [t for t in methods_pudl_out if '_eia861' in t and "etl" not in t]
methods_eia861

['advanced_metering_infrastructure_eia861',
 'balancing_authority_assn_eia861',
 'balancing_authority_eia861',
 'demand_response_eia861',
 'demand_response_water_heater_eia861',
 'demand_side_management_ee_dr_eia861',
 'demand_side_management_misc_eia861',
 'demand_side_management_sales_eia861',
 'distributed_generation_fuel_eia861',
 'distributed_generation_misc_eia861',
 'distributed_generation_tech_eia861',
 'distribution_systems_eia861',
 'dynamic_pricing_eia861',
 'energy_efficiency_eia861',
 'green_pricing_eia861',
 'mergers_eia861',
 'net_metering_customer_fuel_class_eia861',
 'net_metering_misc_eia861',
 'non_net_metering_customer_fuel_class_eia861',
 'non_net_metering_misc_eia861',
 'operational_data_misc_eia861',
 'operational_data_revenue_eia861',
 'reliability_eia861',
 'sales_eia861',
 'service_territory_eia861',
 'utility_assn_eia861',
 'utility_data_misc_eia861',
 'utility_data_nerc_eia861',
 'utility_data_rto_eia861']

### EIA 861 Balancing Authorities

In [49]:
%%time
pudl_out.balancing_authority_eia861().sample(n_samples)

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:37: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo

,report_date,balancing_authority_id_eia,balancing_authority_code_eia,balancing_authority_name_eia
6110,2003-01-01,599,AMPL,"Anchorage, Municipality of"
25010,2009-01-01,99999,NaN,Other
15591,2006-01-01,13756,NaN,Northern Indiana Pub Serv Co
38975,2019-01-01,1,NBSO,New Brunswick System Operator
34027,2011-01-01,19547,HECO,Hawaiian Electric Co Inc


### EIA 861 Advanced Metering Infrastructure

In [50]:
%%time
pudl_out.advanced_metering_infrastructure_eia861().sample(n_samples)

CPU times: user 2.6 ms, sys: 3.97 ms, total: 6.57 ms
Wall time: 6.64 ms


,utility_id_eia,state,balancing_authority_code_eia,report_date,entity_type,short_form,utility_name_eia,customer_class,advanced_metering_infrastructure,automated_meter_reading,daily_digital_access_customers,direct_load_control_customers,energy_served_ami_mwh,home_area_network,non_amr_ami
13345,19219,AL,UNK,2009-01-01,NaN,<NA>,Diverse Power Incorporated,commercial,129,<NA>,<NA>,<NA>,2524.0,<NA>,<NA>
117970,733,WV,PJM,2020-01-01,NaN,<NA>,Appalachian Power Co,commercial,11550,49882,0,0,65790.0,0,78
31015,8566,WY,UNK,2012-01-01,NaN,<NA>,High Plains Power Inc,commercial,0,380,<NA>,<NA>,NaN,<NA>,<NA>
70229,7090,GA,SOCO,2016-01-01,NaN,<NA>,GreyStone Power Corporation,transportation,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>
19599,17874,UT,UNK,2010-01-01,NaN,<NA>,City of St George,transportation,0,0,<NA>,<NA>,0.0,<NA>,<NA>


### EIA 861 Sales
How much electricity did utilities report selling to different types of customers in each year by state?

In [51]:
%%time
pudl_out.sales_eia861().sample(n_samples)

CPU times: user 14.7 ms, sys: 4.05 ms, total: 18.7 ms
Wall time: 19.7 ms


,utility_id_eia,state,report_date,balancing_authority_code_eia,business_model,data_observed,entity_type,service_type,short_form,utility_name_eia,customer_class,customers,sales_mwh,sales_revenue
330758,57067,TX,2016-01-01,ERCO,energy_services,True,NaN,bundled,<NA>,"ENGIE Retail, LLC",other,NaN,NaN,NaN
75975,11824,AK,2004-01-01,UNK,retail,True,NaN,bundled,<NA>,Matanuska Electric Assn Inc,residential,43761.0,393646.0,41838000.0
316475,3420,WA,2016-01-01,AVA,retail,True,NaN,bundled,<NA>,City of Cheney - (WA),transportation,0.0,0.0,0.0
306049,16851,MO,2015-01-01,AECI,retail,True,NaN,bundled,<NA>,SE-MA-NO Electric Coop,industrial,11.0,16786.0,1142000.0
290451,20397,NE,2014-01-01,SWPP,retail,True,NaN,bundled,<NA>,City of West Point - (NE),residential,1483.0,15538.0,1466000.0


### EIA 861 Service Territories
Which counties (with FIPS codes) each utility reported serving in each year.

In [52]:
%%time
pudl_out.service_territory_eia861().sample(n_samples)

CPU times: user 6.2 ms, sys: 43 µs, total: 6.24 ms
Wall time: 5.87 ms


,county,short_form,state,utility_id_eia,utility_name_eia,report_date,state_id_fips,county_id_fips
148474,Searcy,<NA>,AR,14864,Petit Jean Electric Coop Corp,2013-01-01,05,05129
184511,Chaffee,<NA>,CO,15466,Public Service Co of Colorado,2016-01-01,08,08015
213638,Heard,<NA>,GA,3081,Carroll Electric Member Corp - (GA),2019-01-01,13,13149
181476,Mayes,<NA>,OK,10599,"Lake Region Electric Coop, Inc - (OK)",2016-01-01,40,40097
156680,Logan,<NA>,CO,8570,Highline Electric Assn,2014-01-01,08,08075


### Free Memory

In [53]:
del pudl_out

## FERC Form 714
* **NOTE:** Most of the FERC Form 714 tables have not yet been fully processed.
* We have primarily been focused on the historical hourly demand reported by planning areas.
* As with the EIA 861, the full interim ETL will be run as soon as you ask for any FERC 714 table.
* Also as with the EIA 861, if you don't have the [raw FERC 714 input files](https://zenodo.org/record/4127101) cached locally already, they might take a minute to download.

In [54]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)

In [55]:
# here are all of the FERC 714 tables
methods_ferc714 = [t for t in methods_pudl_out if '_ferc714' in t and "etl" not in t]
methods_ferc714

['adjacency_ba_ferc714',
 'demand_forecast_pa_ferc714',
 'demand_hourly_pa_ferc714',
 'demand_monthly_ba_ferc714',
 'description_pa_ferc714',
 'gen_plants_ba_ferc714',
 'id_certification_ferc714',
 'interchange_ba_ferc714',
 'lambda_description_ferc714',
 'lambda_hourly_ba_ferc714',
 'net_energy_load_ba_ferc714',
 'respondent_id_ferc714']

### FERC 714 Respondents
Currently the processing of the hourly planning area demand table exceeds the available memory on this JupyterHub, so the following cells are commented out.

In [56]:
%%time
respondent_id_ferc714 = pudl_out.respondent_id_ferc714()
respondent_id_ferc714.sample(5)

Running the interim FERC 714 ETL process!
Extracting respondent_id_ferc714 from CSV into pandas DataFrame.
Extracting id_certification_ferc714 from CSV into pandas DataFrame.
Extracting gen_plants_ba_ferc714 from CSV into pandas DataFrame.


/home/zane/code/catalyst/pudl/src/pudl/extract/ferc714.py:59: UserWarning: Integration of FERC 714 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


Extracting demand_monthly_ba_ferc714 from CSV into pandas DataFrame.
Extracting net_energy_load_ba_ferc714 from CSV into pandas DataFrame.
Extracting adjacency_ba_ferc714 from CSV into pandas DataFrame.
Extracting interchange_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_hourly_ba_ferc714 from CSV into pandas DataFrame.
Extracting lambda_description_ferc714 from CSV into pandas DataFrame.
Extracting description_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_forecast_pa_ferc714 from CSV into pandas DataFrame.
Extracting demand_hourly_pa_ferc714 from CSV into pandas DataFrame.
Transforming respondent_id_ferc714.
Transforming id_certification_ferc714.
Transforming gen_plants_ba_ferc714.
Transforming demand_monthly_ba_ferc714.
Transforming net_energy_load_ba_ferc714.
Transforming adjacency_ba_ferc714.
Transforming interchange_ba_ferc714.
Transforming lambda_hourly_ba_ferc714.
Transforming lambda_description_ferc714.
Transforming description_pa_ferc714.
Transfor

,respondent_id_ferc714,respondent_name_ferc714,eia_code
43,147,Consumers Energy Company,4254
189,300,NorthWestern Energy (South Dakota),13809
90,198,Madison Gas & Electric Company,11479
152,261,Sunflower Electric Power Corporation,18315
169,280,"Wolverine Power Supply Coop., Inc.",20910


### FERC 714 Hourly Demand by Planning Area

In [57]:
demand_hourly_pa_ferc714 = pudl_out.demand_hourly_pa_ferc714()
demand_hourly_pa_ferc714.sample(20)

,respondent_id_ferc714,report_date,utc_datetime,timezone,demand_mwh
13559459,139,2015-01-01,2015-10-24 04:00:00,America/Los_Angeles,582.45
7364184,152,2008-01-01,2008-06-18 16:00:00,America/New_York,6795.00
3394915,261,2007-01-01,2007-09-23 11:00:00,America/Chicago,236.00
936388,140,2011-01-01,2011-09-30 06:00:00,America/New_York,247.00
2350778,252,2013-01-01,2013-11-25 09:00:00,America/Chicago,864.00
15501814,172,2014-01-01,2014-06-28 04:00:00,America/New_York,279.32
8130661,151,2011-01-01,2011-12-24 18:00:00,America/Chicago,117.00
11706447,274,2018-01-01,2018-08-23 00:00:00,America/Denver,1213.00
6472471,307,2018-01-01,2018-03-28 17:00:00,America/Los_Angeles,8166.00
7853813,274,2020-01-01,2020-12-13 18:00:00,America/Denver,909.00


# Future Analyses
The output object contains a lot of different kinds of things, and as we accumulate more and different kinds of analyses in our library, we're looking to break them out into their own reusable classes that access the database directly. Some work in progress here is related to constructing historical service territory geometries for both utilities and balancing authorities, and associating that data usefully with the FERC 714 respondents. Exploring that in detail is beyond the scope of this notebook, but check out the `pudl.output.ferc714` and `pudl.analysis.service_territory` modules for examples. Unfortunately as this analysis currently depends on both the interim EIA 861 and the interim FERC 714 datasets, it uses too much memory to be run on the JupyterHub right now.

In [58]:
%%time
ferc714_out = pudl.output.ferc714.Respondents(pudl_out)
annualized = ferc714_out.annualize()
categorized = ferc714_out.categorize()
summarized = ferc714_out.summarize_demand()
fipsified = ferc714_out.fipsify()
counties_gdf = ferc714_out.georef_counties()

Running the interim EIA 861 ETL process!
Extracting eia861 spreadsheet data.


/home/zane/code/catalyst/pudl/src/pudl/extract/eia861.py:37: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Transforming raw EIA 861 DataFrames for service_territory_eia861 concatenated across all years.
Assigned state FIPS codes fo